# Indexing voi mo hinh xac suat

In [4]:
import os
import nltk
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from whoosh.index import create_in
from whoosh.fields import *
from whoosh.analysis import StandardAnalyzer
from whoosh import qparser
from whoosh import scoring
import whoosh.index as index

import pytrec_eval
import math

**1. Tien xu ly du lieu**

In [1]:
import py_vncorenlp

In [2]:
def load_vncorenlp(model_dir):
    import os, py_vncorenlp
    
    jar_path = os.path.join(model_dir, "VnCoreNLP-1.2.jar")
    if not os.path.exists(jar_path):
        raise FileNotFoundError(f"❌ Missing {jar_path}")
    
    if not os.path.exists(os.path.join(model_dir, "models")):
        raise FileNotFoundError(f"❌ Missing model folder at {model_dir}/models")
    
    return py_vncorenlp.VnCoreNLP(
        annotators=["wseg", "pos", "ner", "parse"],
        save_dir=model_dir,
        max_heap_size='-Xmx2g'
    )

model = load_vncorenlp(r"C:/Users/mt200/OneDrive/Desktop/AI/InformationRetrieval/Project_InformationRetrieval/Reference/VnCoreNLP-master/")
print(model.word_segment("tôi yêu xử lý ngôn ngữ tự nhiên."))

['tôi yêu xử_lý ngôn_ngữ_tự_nhiên .']


In [3]:
print(model.word_segment("bình ba là một đảo nhỏ diện tích trên 3 km² thuộc xã cam bình, thành phố cam ranh, tỉnh khánh hòa, việt nam."))

['bình ba là một đảo nhỏ diện_tích trên 3 km² thuộc xã cam bình , thành_phố cam_ranh , tỉnh khánh_hoà , việt_nam .']


In [7]:
puncts = ['.', ',', ':', '`', '"', "'", '!', '?', "``", "''"]
stoplist = set()
with open(r"C:\Users\mt200\OneDrive\Desktop\AI\InformationRetrieval\Project_InformationRetrieval\3.Indexing\stopword\vietnamese-stopwords-dash.txt", "r", encoding="utf-8") as f:
    for line in f:
        stoplist.add(line.strip())

In [8]:
def preprocess(tok, punctlist=puncts, stopwords=stoplist):
  tok = tok.lower()
  if tok.isdigit():
    return None
  if tok.isnumeric():
    return None
  if tok in punctlist:
    return None
  if tok in stopwords:
    return None
  return tok

**2. Lap chi muc**

In [18]:
def indexing(src, idx="ind"):
    if not src.endswith('/'):
        src += '/'

    # Ensure index directory exists
    os.makedirs(idx, exist_ok=True)

    schema = Schema(docid=STORED(), content=TEXT(stored=True, analyzer=StandardAnalyzer()))
    ix = create_in(idx, schema)
    writer = ix.writer()

    files = os.listdir(src)
    for f in files:
        file_path = os.path.join(src, f)
        if not os.path.isfile(file_path):
            continue

        encodings_to_try = ['utf-8', 'utf-16', 'cp1252']
        content = None
        for enc in encodings_to_try:
            try:
                with open(file_path, encoding=enc) as r:
                    content = r.read()
                break
            except UnicodeDecodeError:
                continue

        if content is None:
            print(f"⚠️ Skipped file {f}: could not decode with {encodings_to_try}")
            continue

        try:
            terms = []
            for sent in sent_tokenize(content.strip()):
                temp_sent_dash = model.word_segment(sent)

                # ✅ FIX: join list of tokens into one string
                if isinstance(temp_sent_dash, list):
                    temp_sent_dash = " ".join(temp_sent_dash)

                for tok in word_tokenize(temp_sent_dash):
                    tok = preprocess(tok)
                    if tok:
                        terms.append(tok)

            cont = " ".join(terms)
            writer.add_document(docid=f.split(".")[0], content=cont)

        except Exception as e:
            print(f"⚠️ Skipped file {f}: {e}")

    writer.commit()
    print(f"✅ Indexing completed. Index stored in: {idx}")

In [19]:
indexing("C:/Users/mt200/OneDrive/Desktop/AI/InformationRetrieval/Project_InformationRetrieval/1.CollectingDocuments/data_clean", "ind")

⚠️ Skipped file Chinh_phuc_thac_Lieng_Nung_-_ve_ep_sieu_thuc_cua_Tay_Nguyen.txt: JVM exception occurred: java.lang.ArrayIndexOutOfBoundsException
java.lang.ArrayIndexOutOfBoundsException
✅ Indexing completed. Index stored in: ind


**3. Kiem tra mot so truy van**

In [20]:
from whoosh.qparser import QueryParser
from whoosh.index import open_dir
from whoosh import scoring  # ⚡ cần import thêm

def search_query(idx="ind", query_str=""):
    """
    Thực hiện truy vấn trên chỉ mục Whoosh bằng mô hình xác suất BM25.
    - idx: đường dẫn đến thư mục chỉ mục
    - query_str: chuỗi truy vấn (từ khóa người dùng)
    """
    try:
        # Mở thư mục chỉ mục
        ix = open_dir(idx)

        # Tạo parser cho trường 'content'
        qp = QueryParser("content", schema=ix.schema)
        q = qp.parse(query_str)

        # ⚡ Sử dụng mô hình BM25 thay vì TF-IDF
        with ix.searcher(weighting=scoring.BM25F()) as searcher:
            results = searcher.search(q, limit=10)

            if not results:
                print("❌ Không tìm thấy tài liệu phù hợp.")
                return

            print(f"✅ Tìm thấy {len(results)} tài liệu liên quan (BM25):\n")
            for rank, hit in enumerate(results, start=1):
                print(f"{rank}. DocID: {hit['docid']}")
                snippet = hit['content'][:150].replace('\n', ' ')
                print(f"   Trích đoạn: {snippet}...\n")

    except Exception as e:
        print(f"⚠️ Lỗi khi truy vấn: {e}")

In [21]:
search_query("ind", "biển đảo Việt Nam")

✅ Tìm thấy 77 tài liệu liên quan (BM25):

1. DocID: Hon_Tam__Wikipedia_tieng_Viet
   Trích đoạn: hòn tằm ( gọi thuỷ kim_sơn ) hòn đảo rộng ha nằm vịnh nha_trang nằm km đông nam thành_phố hòn đảo bãi biển đẹp nha_trang thu_hút du_lịch tổng_quan đảo...

2. DocID: Cam_nang_du_lich_Kien_Giang
   Trích đoạn: du_lịch cẩm_nang kiên_giang trở_lại du_lịch hướng di_chuyển lưu_trú tham_quan ẩm_thực lưu_ý kiên_giang đồng_bằng sông_cửu_long tỉnh diện_tích tây_nam_...

3. DocID: Cam_nang_du_lich_Nam_Du
   Trích đoạn: du_lịch cẩm_nang nam du trở_lại du_lịch hướng di_chuyển lưu_trú tham_quan ẩm_thực lưu_ý nam du hai xã an sơn nam du huyện kiên_hải diện_tích ha hòn đả...

4. DocID: Cam_nang_du_lich_Co_To
   Trích đoạn: du_lịch cẩm_nang cô_tô trở_lại du_lịch hướng cô_tô mùa đẹp di_chuyển ăn_uống homestay nhà_nghỉ lưu_ý cô_tô huyện đảo đông tỉnh quảng_ninh đất_liền km ...

5. DocID: Cua_Lo_(thi_xa)__Wikipedia_tieng_Viet
   Trích đoạn: x t s cửa_lò thị_xã cũ nằm đông nam tỉnh nghệ_an thị_xã cửa_lò sáp_nhậ

# Danh gia mo hinh

***2) Xử lý truy vấn***
- Chuẩn bị tập Ground Truth
- Chuẩn bị tập câu truy vấn.
- Truy vấn chỉ mục với mô hình xác suất, trọng số term tính theo TF_IDF

In [ ]:
def readGroundTruth(src):
  if src[-1] != '/':
    src += '/'

  GT = {}
  for f in os.listdir(src):
    r = open(src + f)
    rel = {}
    for s in r:
      s = s.strip()
      sp = s.split("\t")
      if len(sp) < 2:
        continue
      did = sp[0].split(" ")[1]
      rel[did] = int(sp[1])
    GT[f.split(".")[0]] = rel
    r.close()
  return GT

In [ ]:
GroundTruth = readGroundTruth("Cranfield/RES")
print(GroundTruth)

In [ ]:
def readQuery(src):
  qry = {}
  r = open(src)
  for s in r:
    s = s.strip()
    ps = s.split("\t")
    terms = []
    for sent in sent_tokenize(ps[1]):
      for tok in word_tokenize(sent):
        tok = preprocess(tok)
        if tok != None:
          terms.append(tok)
    qry[ps[0]] = " ".join(terms)
  return qry

In [ ]:
Queries = readQuery("Cranfield/query.txt")
print(Queries)

In [ ]:
def processQueries(ind, qry):

  idx = index.open_dir(ind)
  searcher = idx.searcher(weighting=scoring.BM25F())
  parser = qparser.QueryParser("content", idx.schema, group=qparser.OrGroup)

  RET = {}
  for key in qry:
    query = parser.parse(qry[key])
    results = searcher.search(query, limit=None)
    rel = {}
    for i in range(len(results)):
      rel[results[i]["docid"]] = results[i].score
    RET[key] = rel
  return RET

In [ ]:
RunResults = processQueries("ind", Queries)

***3) Tính chỉ số***
- Sử dụng đối tượng thuộc lớp RelevanceEvaluator từ package pytrec_eval.
- Tính chỉ số MAP ("intAP") và MAPint ("11pt_avg")

In [ ]:
print(pytrec_eval.supported_measures)
eval = pytrec_eval.RelevanceEvaluator(GroundTruth, ["infAP", "11pt_avg"])
Results = eval.evaluate(RunResults)

MAP = 0
MAP11PT = 0

for key in Results:
  value = Results[key]["infAP"]
  if not math.isnan(value):
    MAP += value
  value = Results[key]["11pt_avg"]
  if not math.isnan(value):
    MAP11PT += value

MAP /= len(Results)
MAP11PT /= len(Results)

print(MAP, MAP11PT)